In [ ]:
import kaggle
!kaggle competitions download -c spaceship-titanic

In [141]:
import h2o
h2o.init()
trainRaw = h2o.import_file('train.csv')
testRaw = h2o.import_file('test.csv')

trainRaw.describe()


Checking whether there is an H2O instance running at http://localhost:54321.. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,1 day 23 hours 45 mins
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,3 months and 25 days
H2O_cluster_name:,H2O_from_python_kevin_qgh2wf
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.887 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Rows:8693
Cols:14

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
type,string,enum,enum,enum,enum,int,enum,int,int,int,int,int,string,enum
mins,NaN,,,,,0.0,,0.0,0.0,0.0,0.0,0.0,NaN,
mean,NaN,,,,,28.82793046746535,,224.68761748120303,458.07720329024676,173.72916912197996,311.1387779083431,304.8547912992357,NaN,
maxs,NaN,,,,,79.0,,14327.0,29813.0,23492.0,22408.0,24133.0,NaN,
sigma,NaN,,,,,14.48902142390878,,666.7176629280652,1611.489240355072,604.6964584708243,1136.7055348344065,1145.7171888056614,NaN,
zeros,0,,,,,178,,5577,5456,5587,5324,5495,0,
missing,0,201,217,199,182,179,203,181,183,208,183,188,200,0
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False


In [ ]:
import pandas as pd
import numpy as np
trainRawDF = pd.read_csv('train.csv')
testRawDF = pd.read_csv('test.csv')
def curate(df):
    df[['Deck', 'CabinNum', 'Side']] = df['Cabin'].str.split('/', expand=True, n=3)
    df['CabinNumLen'] = df['CabinNum'].str.len()
    df['CabinNum'] = pd.to_numeric(df['CabinNum'], errors='coerce')
    df['CabinRegion'] = pd.qcut(df['CabinNum'], q=7)
    df['AgeDecile'] = pd.qcut(df['Age'], q=10)

    df[['FirstName', 'LastName']] = df['Name'].str.split(' ', expand=True, n=2)
    df['GroupNum'] = df['PassengerId'].apply(lambda x: x.split('_')[0]).astype(int)

    df['FamilySize'] = df.groupby(['LastName'])['LastName'].transform('size')
    df['GroupSize'] = df.groupby(['GroupNum'])['GroupNum'].transform('size')
    df['CabinSize'] = df.groupby(['CabinNum'])['CabinNum'].transform('size')

    df['GroupSize'] = df.groupby(['GroupNum'])['GroupNum'].transform('size')
    df['CabinSize'] = df.groupby(['CabinNum'])['CabinNum'].transform('size') 

    df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].fillna(0, inplace=True)
    df['Expenditure'] = df['RoomService'] + df['FoodCourt'] + df['ShoppingMall'] + df['Spa'] + df['VRDeck']
    df['LogExpenditure'] = np.log(df['Expenditure'] + 1)
    df['ZeroExpense'] = df['Expenditure'] == 0
    return df

trainProcessedDF = curate(trainRawDF)
testProcessedDF = curate(testRawDF)

print(trainProcessedDF.shape)
trainProcessedDF.describe()

In [ ]:
def countNan(df):
    nan_counts_column = df.isna().sum()
    print(nan_counts_column)

def impute(df):
    # HOME_PLANET
    # Decks A, B, C or T came from Europa. Deck G came from Earth
    df.loc[(df['HomePlanet'].isna()) & (df['Cabin_deck'].isin(['A', 'B', 'C', 'T'])), 'HomePlanet']='Europa'
    df.loc[(df['HomePlanet'].isna()) & (df['Cabin_deck']=='G'), 'HomePlanet']='Earth'
    # Surnames have the same HomePlanet
    SHP_gb=df.groupby(['Surname','HomePlanet'])['HomePlanet'].size().unstack().fillna(0)
    SHP_index=df[df['HomePlanet'].isna()][(df[df['HomePlanet'].isna()]['Surname']).isin(SHP_gb.index)].index
    df.loc[SHP_index,'HomePlanet']=df.iloc[SHP_index,:]['Surname'].map(lambda x: SHP_gb.idxmax(axis=1)[x])
    # For the rest, let's set source=Earth
    df.loc[(df['HomePlanet'].isna()), 'Destination']='Earth'

    df.loc[(df['Destination'].isna()), 'Destination']='TRAPPIST-1e'

    # NAME / FAMILY SIZE - what does this code do..?
    GSN_gb=df[df['Group_size']>1].groupby(['Group','Surname'])['Surname'].size().unstack().fillna(0)

    df['Family_size']=df['Surname'].map(lambda x: df['Surname'].value_counts()[x])
    df.loc[df['Surname']=='Unknown','Surname']=np.nan
    df.loc[df['Family_size']>100,'Family_size']=0
    GSN_index=df[df['Surname'].isna()][(df[df['Surname'].isna()]['Group']).isin(GSN_gb.index)].index
    df.loc[GSN_index,'Surname']=df.iloc[GSN_index,:]['Group'].map(lambda x: GSN_gb.idxmax(axis=1)[x])

    # CABIN
    GCD_gb=df[df['Group_size']>1].groupby(['Group','Cabin_deck'])['Cabin_deck'].size().unstack().fillna(0)
    GCN_gb=df[df['Group_size']>1].groupby(['Group','Cabin_number'])['Cabin_number'].size().unstack().fillna(0)
    GCS_gb=df[df['Group_size']>1].groupby(['Group','Cabin_side'])['Cabin_side'].size().unstack().fillna(0)

    # Passengers with missing cabin side, in a group with known cabin side.
    GCS_index=df[df['Cabin_side'].isna()][(df[df['Cabin_side'].isna()]['Group']).isin(GCS_gb.index)].index
    df.loc[GCS_index,'Cabin_side']=df.iloc[GCS_index,:]['Group'].map(lambda x: GCS_gb.idxmax(axis=1)[x])

    # VIP = Mode
    df.loc[df['VIP'].isna(),'VIP']=False

    # AGE = median of each subgroup
    na_rows_A=df.loc[df['Age'].isna(),'Age'].index
    df.loc[df['Age'].isna(),'Age']=df.groupby(['HomePlanet','No_spending','Solo','Cabin_deck'])['Age'].transform(lambda x: x.fillna(x.median()))[na_rows_A]

    # CRYOSLEEP
    df['CryoSleep']
    na_rows_CSL=df.loc[df['CryoSleep'].isna(),'CryoSleep'].index
    df.loc[df['CryoSleep'].isna(),'CryoSleep']=df.groupby(['No_spending'])['CryoSleep'].transform(lambda x: x.fillna(pd.Series.mode(x)[0]))[na_rows_CSL]

    # EXPENDITURES
    for col in exp_feats:
        data.loc[(data[col].isna()) & (data['CryoSleep']==True), col]=0

    df['RoomService']
    df['FoodCourt']
    df['ShoppingMall']
    df['Spa']
    df['VRDeck']

    return df

trainRawDF = pd.read_csv('train.csv')
testRawDF = pd.read_csv('test.csv')

countNan(trainRawDF)
trainProcessedDF = impute(trainRawDF)
countNan(trainRawDF)

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64
PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64


In [ ]:
corr_matrix = trainProcessedDF[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].corr()
print(corr_matrix)
print(trainProcessedDF.nunique())
print(trainProcessedDF.dtypes)
print(trainProcessedDF.head())

In [ ]:
import altair as alt
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt


In [ ]:
sns.pairplot(trainRawDF, hue='Transported', kind='kde', corner=True)

In [ ]:
def numericScatterPlots(df, numericList, colorLabel):
    comboList = []
    for i in range(len(numericList)):
        for j in range(i+1, len(numericList)):
            comboList.append((numericList[i], numericList[j]))

    numCombos = len(comboList)

    fig=plt.figure(figsize=(10,numCombos*4))
    for i, (x, y) in enumerate(comboList):
        ax=fig.add_subplot(numCombos,1,i+1)
        sns.scatterplot(data=df, x=x, y=y, hue=colorLabel)
        ax.set_title(f'{x} vs {y}') 
    fig.tight_layout()

def categoricalCountPlots(df, categoricalList, colorLabel):
    numFeats = len(categoricalList)
    fig=plt.figure(figsize=(10,numFeats*4))
    for i, var_name in enumerate(categoricalList):
        ax=fig.add_subplot(numFeats,1,i+1)
        sns.countplot(data=df, x=var_name, axes=ax, hue=colorLabel)
        ax.set_title(var_name) 
    fig.tight_layout()

In [ ]:
import sys
import os
wd = os.getcwd()
print(wd)
sys.path.insert(0, wd)
                
import plotHelper as ph
categoricalList=['HomePlanet', 'Destination', 'CryoSleep', 'VIP', 'ZeroExpense', 'GroupSize', 'FamilySize', 'CabinSize']
categoricalList.extend(['CabinRegion', 'AgeDecile', 'Deck','Side', 'CabinNumLen']) # Cabin derived features
categoricalCountPlots(trainProcessedDF, categoricalList, 'Transported')

In [ ]:
numerics=['CabinNum', 'GroupNum', 'Expenditure', 'LogExpenditure', 'Age']
numericTuples=[('CabinNum', 20), ('GroupNum', 200), ('Expenditure',3000), ('LogExpenditure', 0.1), ('Age', 1)]

def numericHistograms(df, numericList, colorLabel):
    numNumerics = len(numericList)
    fig=plt.figure(figsize=(10,numNumerics*4))
    for i, (numeric, bw) in enumerate(numericList):
        ax=fig.add_subplot(numNumerics,1,i+1)
        sns.histplot(data=df, x=numeric, hue=colorLabel, binwidth=bw, kde=True)
        ax.set_title(numeric) 
    fig.tight_layout()
numericHistograms(trainProcessedDF, numericTuples, 'Transported')

In [ ]:
def catCombo_Heatmap(df, comboList):
    numCombos = len(comboList)
    fig=plt.figure(figsize=(10,numCombos*4))
    for i, (x, y) in enumerate(comboList):
        gbdf=df.groupby([y, x])[x].size().unstack().fillna(0)
        ax=fig.add_subplot(numCombos,1,i+1)
        sns.heatmap(gbdf.T, annot=True, fmt='g', cmap='coolwarm')
        ax.set_title(f'{x} vs {y}') 
    fig.tight_layout()

combos = [('HomePlanet', 'Destination'), ('FamilySize', 'GroupSize'), ('CabinSize', 'GroupSize'), ('CabinSize', 'FamilySize')]
catCombo_Heatmap(trainProcessedDF, combos)

In [ ]:
"""For each family size, what is the PCTOFTOTAL of Transported?
    FamSz=2: Pct=0, 0.5, 1.0
    FamSz=3: Pct=[0,0.3, 0.6, 1.0]
    FamSz=4: Pct=[0,0.25, 0.5, 0.75, 1.0]

    [FamilySize][PercentOfTotal].size()
    PctTotal is a groupbyFamily.
    Count PctTotal is a groupby FamilySize
"""

trainProcessedDF['Ysum'] = trainProcessedDF.groupby(['LastName'])['Transported'].transform('sum')
trainProcessedDF['Ysize'] = trainProcessedDF.groupby(['LastName'])['Transported'].transform('size')
trainProcessedDF['Ypct'] = trainProcessedDF['Ysum'].div(trainProcessedDF['Ysize']).astype('str')
gb = trainProcessedDF.groupby(['LastName'])['Ypct'].avg()
print(trainProcessedDF[['LastName', 'FamilySize', 'Transported', 'Ypct', 'Ysum', 'Ysize']].sort_values(by="LastName").head(n=60))



In [ ]:
def catCombo_GbCountplot(df, comboList):
    dfDict = {}
    for tpl in comboList:
        (y,x) = tpl
        catCountDF = df.groupby([y, x])[x].size().unstack().fillna(0)
        uniqueSeries = (catCountDF>0).sum(axis=1).reset_index(name='Count')['Count'].sort_values().astype(str)
        dfDict[tpl] = uniqueSeries
    numCombos = len(comboList)

    for tpl, df in dfDict.items():
        print(f"{tpl} -> {df.shape}")

    fig=plt.figure(figsize=(10,numCombos*4))
    for i, (tpl, dfgb) in enumerate(dfDict.items()):
        (x,y) = tpl
        ax=fig.add_subplot(numCombos,1,i+1)
        sns.countplot(x=dfgb)
        ax.set_title(f"Unique {y} per {x}")
    fig.tight_layout()

combos = [('GroupNum','Deck'), ('GroupNum', 'CabinNum'), ('GroupNum', 'Side'), ('GroupNum', 'HomePlanet'), ('GroupNum', 'Destination'), ('GroupNum', 'CryoSleep')]
filteredDF = trainProcessedDF[trainProcessedDF['GroupSize'] > 1]
catCombo_GbCountplot(filteredDF, combos)

Conclusions:
* Group members always share the same HomePlanet, and the same side.
* Group members can have different cabins, decks, destinations, and cryosleep status.


In [ ]:
combos = [('LastName','Deck'), ('LastName', 'CabinNum'), ('LastName', 'Side'), ('LastName', 'HomePlanet'), ('LastName', 'Destination'), ('LastName', 'CryoSleep')]
# filteredDF = trainProcessedDF[trainProcessedDF['FamilySize'] > 1]
catCombo_GbCountplot(filteredDF, combos)

Conclusions:
* Family Members always come from the same home planet.
* But they can differ among other attributes. 

In [ ]:
import pygwalker as pyg
pyg.walk(trainProcessedDF)

In [ ]:
import h2o
h2o.init()

trainHF = h2o.H2OFrame(trainProcessedDF)
testHF = h2o.H2OFrame(testProcessedDF)

trainHF.describe()
x = trainHF.columns
y = "Transported"
trainHF[y] = trainHF[y].asfactor()
x.remove(y)


from h2o.automl import H2OAutoML
aml = H2OAutoML(max_models=10, seed=42, max_runtime_secs=7200, sort_metric='accuracy')
aml.train(x=x, y=y, training_frame=trainHF)
print(aml.leaderboard)

# Raw: Accuracy 79.1 - 80.1%
# Curated: Accuracy 79.4 - 80.9%

In [ ]:
print(aml.leader.varimp(use_pandas=True))
aml.leader.varimp_plot()